In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

24/05/06 12:40:58 WARN Utils: Your hostname, aliyu-XPS-15-9550 resolves to a loopback address: 127.0.1.1; using 192.168.199.55 instead (on interface wlp2s0)
24/05/06 12:40:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 12:40:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/06 12:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/06 12:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/06 12:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/06 12:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/05/06 12:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attemptin

In [3]:
data_path = './Data'

In [4]:
json_df2_path = data_path + '/utilization.json'
df_util = spark.read.format('json').load(json_df2_path)

In [5]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [6]:
df_util.createOrReplaceTempView('utilization')

In [7]:
df_util.count()

500000

In [8]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization),stddev(cpu_utilization) \
                        FROM utilization \
                        GROUP BY server_id').show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      103|                0.56|                0.96|    0.11617507884178278|
|      104|                0.51|                0.91|    0.11521679513850511|
|      106|                0.22|                0.62|    0.11531539914568226|
|      100|                0.27|                0.67|     0.1152264191787964|
|      105|                0.29|                0.69|    0.11510721467869486|
|      101|                 0.6|                 1.0|    0.11651726263197697|
|      102|                0.56|                0.96|    0.11549678751286807|
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      110|                0.35|                0.75|    0.11533

In [10]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, avg(cpu_utilization) \
                    OVER (PARTITION BY server_id) avg_server_util \
                    FROM utilization').show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:46:31|      110|           0.37|0.5537749999999892|
|03/05/2019 08:51:31|      110|            0.7|0.5537749999999892|
|03/05/2019 08:56:31|      110|           0.67|0.5537749999999892|
|03/05/2019 09:01:31|      110|           0.56|0.5537749999999

In [11]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, avg(cpu_utilization) \
                    OVER (PARTITION BY server_id) avg_server_util \
                    FROM utilization')

DataFrame[event_datetime: string, server_id: bigint, cpu_utilization: double, avg_server_util: double]

In [12]:
sql_window = spark.sql('SELECT event_datetime, server_id, cpu_utilization, avg(cpu_utilization) \
                    OVER (PARTITION BY server_id) avg_server_util \
                    FROM utilization')

In [13]:
sql_window.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:46:31|      110|           0.37|0.5537749999999892|
|03/05/2019 08:51:31|      110|            0.7|0.5537749999999892|
|03/05/2019 08:56:31|      110|           0.67|0.5537749999999892|
|03/05/2019 09:01:31|      110|           0.56|0.5537749999999

In [14]:
sql_window2 = spark.sql('SELECT event_datetime, server_id, cpu_utilization, avg(cpu_utilization) \
                    OVER (PARTITION BY server_id) avg_server_util, cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id)delta_server_util \
                    FROM utilization')

In [15]:
sql_window2.show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|  0.1262250000000108|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|0.026225000000010712|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892| 0.15622500000001072|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892| 0.11622500000001079|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:46:31|      110| 

In [17]:
sql_window3 = spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
                avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_server_util \
                FROM utilization')

In [18]:
sql_window3.show()

+-------------------+---------+---------------+-------------------+
|     event_datetime|server_id|cpu_utilization|    avg_server_util|
+-------------------+---------+---------------+-------------------+
|03/05/2019 08:06:31|      110|           0.68|               0.63|
|03/05/2019 08:11:31|      110|           0.58| 0.6033333333333334|
|03/05/2019 08:16:31|      110|           0.55| 0.5866666666666666|
|03/05/2019 08:21:31|      110|           0.63| 0.6033333333333334|
|03/05/2019 08:26:31|      110|           0.63| 0.6566666666666666|
|03/05/2019 08:31:31|      110|           0.71| 0.6699999999999999|
|03/05/2019 08:36:31|      110|           0.67| 0.6433333333333333|
|03/05/2019 08:41:31|      110|           0.55| 0.5300000000000001|
|03/05/2019 08:46:31|      110|           0.37|               0.54|
|03/05/2019 08:51:31|      110|            0.7|               0.58|
|03/05/2019 08:56:31|      110|           0.67| 0.6433333333333334|
|03/05/2019 09:01:31|      110|           0.56| 